In [1]:
import numpy as np
import os

layerList = ["가", "나", "다", "라", "마", "바", "아", "자", "차", "카", "타", "파", "하"]

all_x_array = []
all_y_array = []

for dir in layerList:
    fileList = os.listdir(f"../dataset_dir/image_npy/{dir}")

    for file in fileList:
        y_array = dict.fromkeys(layerList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성

        if file[-8:-4] == "data":
            x_array = np.load(f"../dataset_dir/image_npy/{dir}/{file}")
            all_x_array.append(x_array.tolist())

            y_array[dir] = 1 # 사용자가 지정한 str 값을 1로 변경
            y_array = list(y_array.values())
            all_y_array.append(y_array)


In [2]:
x_array = np.array(all_x_array)
y_array = np.array(all_y_array)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array, y_array, test_size=0.4, shuffle=False, random_state=1004
)

In [4]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [5]:
image_w, image_h = X_train.shape[1], X_train.shape[2]

X_train = X_train.reshape(-1, image_w, image_h, 1)
X_test = X_test.reshape(-1, image_w, image_h, 1)

In [6]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(476, 200, 200, 1) (318, 200, 200, 1)
(476, 13) (318, 13)


In [7]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus: # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    print(f"{gpus[0]} 해당 GPU가 할당 되었습니다.")
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

2024-02-25 05:35:24.845495: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-25 05:35:24.876329: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 05:35:24.876368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 05:35:24.877382: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 05:35:24.883229: I tensorflow/core/platform/cpu_feature_guar

In [8]:
import tensorflow as tf
import keras

model = keras.Sequential([
    keras.Input(shape=(200, 200, 1)),
    # keras.layers.Dense(1250, activation='relu'),
    # keras.layers.Dense(625, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(512, activation='relu'),
    # keras.layers.Dense(156, activation='relu'),
    # keras.layers.Dense(78, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    # keras.layers.Dense(19, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(32, activation='relu'),
    # keras.layers.Dense(19, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(13, activation='softmax'),
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 200, 200, 1)       0         
                                                                 
 dense (Dense)               (None, 200, 200, 512)     1024      
                                                                 
 dropout_1 (Dropout)         (None, 200, 200, 512)     0         
                                                                 
 dense_1 (Dense)             (None, 200, 200, 64)      32832     
                                                                 
 dropout_2 (Dropout)         (None, 200, 200, 64)      0         
                                                                 
 dense_2 (Dense)             (None, 200, 200, 32)      2080      
                                                                 
 dropout_3 (Dropout)         (None, 200, 200, 32)      0

In [10]:
batch_size = 16
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.05), metrics=["accuracy"])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

Epoch 1/5
10/30 [=========>....................] - ETA: 35s - loss: 1632.5060 - accuracy: 0.1187